# Sentence Variational Adversarial Active Learning (S-VAAL)
@author: Tyler Bikaun

The following notebook intends to flesh out an initial proof concept of the coupling of S-VAE (Bowman <i>et al.</i> 2016; https://arxiv.org/abs/1511.06349) and VAAL (Sinha <i>et al.</i> 2019; https://arxiv.org/abs/1904.00370)

<b>Application:</b> Named Entity Task (NER)

### Standard Imports

In [57]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torch.nn.utils.rnn as rnn_utils

### Utility Functions

- [ ] Build data preprocessor
- [ ] Build data loaders

In [ ]:
config = {'': ''}

### Models
<i>Model architectures</i><br>
<b>SVAE</b> - RNN<br>
<b>Discriminator</b> - FC NN<br>
<b>Task Learner</b> - RNN<br>
- [ ] SVAE
- [ ] Discriminator
- [ ] Task Learner

#### SVAE
To do:
 - [ ] 

In [284]:
class SVAE(nn.Module):
    """ Sentence Variational Autoencoder (Bowman et al. 2016)"""
    
    def __init__(self, vocab_size, embedding_size):
        super(SVAE, self).__init__()
        
        # TODO: fix dodgy vocab_size issue... this will be cleared up when utils implemented properly
        
        self.tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.Tensor
        
        self.max_sequence_length = 40    # arg
        self.pad_idx = 0
        self.eos_idx = vocab_size + 1
        self.sos_idx = vocab_size + 2
        self.unk_idx = vocab_size + 3
        
        self.vocab_size = vocab_size + 4
        
        self.z_dim = 8
        
        self.rnn_type = 'gru'
        self.bidirectional = False
        self.num_layers = 1
        self.hidden_size = 128
        
        self.embedding = nn.Embedding(self.vocab_size, embedding_size)
        self.word_dropout_rate = 0.1
        self.embedding_dropout = nn.Dropout(p=0.5)
        
        # set rnn type
        if self.rnn_type == 'gru':
            rnn = nn.GRU
        else:
            raise ValueError()
        
        # init encoder-decoder RNNs (models are identical)
        self.encoder_rnn = rnn(embedding_size,
                               self.hidden_size, 
                               num_layers=self.num_layers,
                               bidirectional=self.bidirectional,
                               batch_first=True)
        self.decoder_rnn = rnn(embedding_size,
                               self.hidden_size, 
                               num_layers=self.num_layers,
                               bidirectional=self.bidirectional,
                               batch_first=True)

        self.hidden_factor = (2 if self.bidirectional else 1) * self.num_layers
        
        # Initialisation of FC layers
        # These go from encoder to latent (z) space
        self.hidden2mean = nn.Linear(self.hidden_size * self.hidden_factor, self.z_dim)
        self.hidden2logv = nn.Linear(self.hidden_size * self.hidden_factor, self.z_dim)
        self.z2hidden = nn.Linear(self.z_dim, self.hidden_size * self.hidden_factor)
        self.outputs2vocab = nn.Linear(self.hidden_size * (2 if self.bidirectional else 1), self.vocab_size)
    
    
    def forward(self, input_sequence, length):
        """ Forward pass through VAE """
        
        batch_size = input_sequence.size(0)
        sorted_lengths, sorted_idx = torch.sort(length, descending=True)   # trick for packed padding
        input_sequence = input_sequence[sorted_idx]
        
        # ENCODER
        input_embedding = self.embedding(input_sequence)
#         print(input_embedding.shape)
        packed_input = rnn_utils.pack_padded_sequence(input_embedding, sorted_lengths.data.tolist(), batch_first=True)
        _, hidden = self._encode(packed_input)
        
        if self.bidirectional or 1 < self.num_layers:
            # flatten hidden state
            hidden = hidden.view(batch_size, self.hidden_size * self.hidden_factor)
        else:
            # .squeeze() -> Returns a tensor with all the dimensions of input of size 1 removed.
            print(f'hidden shape before squeeze {hidden.shape}')
#             hidden = hidden.squeeze()   # doesn't work? gives wrong dimension down stream...
            pass
            print(f'hidden shape after squeeze {hidden.shape}')

        
        # Reparameterisation trick!
        z, mean, logv, std = self.reparameterise(hidden, batch_size)
        
        # DECODER
        if 0 < self.word_dropout_rate:
            prob = torch.rand(input_sequence.size())

            if torch.cuda.is_available():
                prob = prob.cuda()

            prob[(input_sequence.data - self.sos_idx) * (input_sequence.data - self.pad_idx) == 0] = 1

            decoder_input_sequence = input_sequence.clone()
            
#             print(vocab_size)
#             print(self.unk_idx)
            decoder_input_sequence[prob < self.word_dropout_rate] = self.unk_idx

#             print(decoder_input_sequence)
            input_embedding = self.embedding(decoder_input_sequence)

        input_embedding = self.embedding_dropout(input_embedding)
        packed_input = rnn_utils.pack_padded_sequence(input_embedding, sorted_lengths.data.tolist(), batch_first=True)
        
        outputs, _ = self._decode(packed_input, hidden)
        
        # process outputs
        # Process outputs
        # Unpack padded sequence
        padded_outputs = rnn_utils.pad_packed_sequence(outputs, batch_first=True)[0]
        padded_outputs = padded_outputs.contiguous()
        _, reversed_idx = torch.sort(sorted_idx)
        padded_outputs = padded_outputs[reversed_idx]
        b, s, _ = padded_outputs.size()

        # Project outputs to vocab
        # e.g. project hidden state into label space...
        logp = nn.functional.log_softmax(self.outputs2vocab(padded_outputs.view(-1, padded_outputs.size(2))), dim=-1)
#         print(f'logp before view {logp.shape}\n')
#         print(f'b {b} s {s} no emb {self.embedding.num_embeddings}')
        logp = logp.view(b, s, self.embedding.num_embeddings)

        # logp - log posterior over label space; mean - tensor Gaussian mean, logv - tensor Gaussian variance, z - VAE latent space 
        return logp, mean, logv, z
    
    def to_var(self, x):
        if torch.cuda.is_available():
            x = x.cuda()
        return x
    
    def reparameterise(self, hidden, batch_size):
        """ Implement reparameterisation trick (Kingma and Welling 2014) """
        
        mean = self.hidden2mean(hidden)
        logv = self.hidden2logv(hidden)
        std = torch.exp(0.5 * logv) 
        
        z = self.to_var(torch.randn([batch_size, self.z_dim]))
        return z * std + mean, mean, logv, std
    
    def _encode(self, x):
        """ x - pack padded sequence """
        return self.encoder_rnn(x)
    
    def _decode(self, x, hidden):
        """ x - pack padded sequence
            hidden - latent tensor"""
        return self.decoder_rnn(x, hidden)

In [285]:
# Testing functionality
vocab_size = 100
hidden_size = 128
SVAE = SVAE(vocab_size, hidden_size)

In [286]:
# Test forward pass of SVAE
import random
import numpy as np

# generate some fake sequences with padding
seqs = list()
for i in range(10):
    seq = np.random.randint(low=1, high=100, size=(random.randint(1,40),))
    # add padding
    seq = np.concatenate((seq, np.zeros(shape=(40 - len(seq)))), axis=None)
    seqs.append(seq)
sequences = torch.LongTensor(seqs)
# get lengths of sequences (not including padding)
lengths = torch.tensor([len(seq[seq != 0]) for seq in sequences])

print(f'Shapes - seq {sequences.shape} - lengths {lengths.shape}')
# print(lengths)

# Pass sequences and lengths into SVAE forward method
print(SVAE.forward(sequences, lengths))

Shapes - seq torch.Size([10, 40]) - lengths torch.Size([10])
hidden shape before squeeze torch.Size([1, 10, 128])
hidden shape after squeeze torch.Size([1, 10, 128])
(tensor([[[-4.7107, -4.2747, -5.0126,  ..., -4.7662, -4.6241, -4.4391],
         [-4.7077, -4.2380, -4.5522,  ..., -4.8695, -4.3438, -4.6424],
         [-4.7907, -4.4818, -4.7881,  ..., -4.7656, -4.8037, -4.6855],
         ...,
         [-4.5724, -4.5642, -4.5951,  ..., -4.6142, -4.6185, -4.6610],
         [-4.5724, -4.5642, -4.5951,  ..., -4.6142, -4.6185, -4.6610],
         [-4.5724, -4.5642, -4.5951,  ..., -4.6142, -4.6185, -4.6610]],

        [[-4.6742, -4.5768, -4.4621,  ..., -4.7236, -4.4771, -4.3558],
         [-4.6727, -4.6019, -4.4895,  ..., -4.8585, -4.6749, -4.6464],
         [-4.8492, -4.4185, -4.6465,  ..., -4.9295, -4.5632, -4.5648],
         ...,
         [-4.5724, -4.5642, -4.5951,  ..., -4.6142, -4.6185, -4.6610],
         [-4.5724, -4.5642, -4.5951,  ..., -4.6142, -4.6185, -4.6610],
         [-4.5724, -4.

In [287]:
# Reviewing components
print(f'Encoder: {SVAE.encoder_rnn}')
print(f'Decoder: {SVAE.decoder_rnn}')
print(f'hidden2mean: {SVAE.hidden2mean}')
print(f'hidden2logv: {SVAE.hidden2logv}')
print(f'z2hidden: {SVAE.z2hidden}')
print(f'outputs2vocab: {SVAE.outputs2vocab}')

Encoder: GRU(128, 128, batch_first=True)
Decoder: GRU(128, 128, batch_first=True)
hidden2mean: Linear(in_features=128, out_features=8, bias=True)
hidden2logv: Linear(in_features=128, out_features=8, bias=True)
z2hidden: Linear(in_features=8, out_features=128, bias=True)
outputs2vocab: Linear(in_features=128, out_features=104, bias=True)


#### Task Learner
Using PyTorch tutorial implementation (https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html) for rapid development (will implement SoTA in the future)<br><br>
To do:
 - [ ] 

In [269]:
class TaskLearner(nn.Module):
    """ Task learner for NER """
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(TaskLearner, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [271]:
# Testing functionality
tasklearner = TaskLearner(embedding_dim=128, hidden_dim=128, vocab_size=104, tagset_size=4)

In [274]:
# Testing forward functionality
tasklearner.forward(torch.randint(0,100,size=(10,)))

tensor([[-1.3646, -1.4012, -1.3667, -1.4135],
        [-1.3912, -1.3452, -1.3913, -1.4189],
        [-1.3578, -1.4526, -1.3384, -1.4003],
        [-1.4360, -1.3843, -1.3061, -1.4240],
        [-1.4032, -1.3590, -1.3353, -1.4517],
        [-1.4705, -1.3100, -1.3063, -1.4715],
        [-1.4036, -1.3782, -1.2292, -1.5619],
        [-1.2345, -1.5217, -1.3853, -1.4253],
        [-1.3498, -1.4740, -1.3688, -1.3574],
        [-1.3601, -1.4565, -1.4957, -1.2510]], grad_fn=<LogSoftmaxBackward>)

#### Discriminator
To do:
 - [ ] 

In [42]:
class Discriminator(nn.Module):
    """ Adversary architecture for discriminator module """
    
    def __init__(self, z_dim=8):
        super(Discriminator, self).__init__()
        
        self.z_dim = z_dim    # latent space dimension
        
        self.net = nn.Sequential(
                                nn.Linear(z_dim, 128),
                                nn.ReLU(True),
                                nn.Linear(128, 128),
                                nn.ReLU(True),
                                nn.Linear(128,1),
                                nn.Sigmoid()
                                )
        # Exe
        self.weight_init()
        
    def weight_init(self):
        """ Weight initialisation
        
        Using Xavier uniform initialisation rather than Kaiming (I think that is more focused for CV? TODO: investigate)
        See: https://pytorch.org/cppdocs/api/function_namespacetorch_1_1nn_1_1init_1ace282f75916a862c9678343dfd4d5ffe.html
        """
        for block in self._modules:
            for m in self._modules[block]:
                if type(m) == nn.Linear:
                    torch.nn.init.xavier_uniform_(m.weight)
                    m.bias.data.fill_(0.01)
    
    def forward(self, z):
        """ Forward pass through discriminator
        
        Arguments
        --------
            z : tensor
                Tensor derived from SVAE latent space
        """
        return self.net(z)

In [56]:
# Testing functionality
dsc = Discriminator()

# Pass random tensor through forward pass of discriminator
rand_tensor = torch.randn((8,))
print(f'Input: {rand_tensor}')
print(f'Output: {dsc.forward(rand_tensor)}')

Input: tensor([ 0.0312,  1.0590,  0.3862, -0.1136, -1.1375,  1.2425, -1.3496, -0.7329])
Output: tensor([0.5398], grad_fn=<SigmoidBackward>)


## Sampler Routine
Active learning based sample selection for task learner

In [ ]:
# Code copied from VAAL - TODO: investigate applicability; it may not work for sequence based samples.
class Sampler:
    """ Adversary sampler """
    def __init__(self, budget):
        self.budget = budget
        
    def sample(self, vae, discriminator, data, cuda):
        """ Selective sampling algorithm
        
        Arguments
        ---------
            vae : torch model
                VAE model
            discriminator : torch model
                discriminator model
            data : tensor
                Image data
            cuda : boolean
                GPU flag
        Returns
        -------
            querry_pool_indices: int, list
                List of indices corresponding to sorted (top-K) samples to be sampled from
        """
        all_preds = []
        all_indices = []

        for images, _, indices in data:
            if cuda:
                images = images.cuda()

            with torch.no_grad():
                _, _, mu, _ = vae(images)
                preds = discriminator(mu)

            preds = preds.cpu().data
            all_preds.extend(preds)
            all_indices.extend(indices)

        all_preds = torch.stack(all_preds)
        all_preds = all_preds.view(-1)
        # need to multiply by -1 to be able to use torch.topk 
        all_preds *= -1

        # select the points which the discriminator things are the most likely to be unlabeled
        _, querry_indices = torch.topk(all_preds, int(self.budget))
        querry_pool_indices = np.asarray(all_indices)[querry_indices]

        return querry_pool_indices

### Training Routine
<i> Pseudo code</i>
```python
for epoch in max_epochs:
        train(task learner)
            get preds
            calc loss
            zero grads
            backpropigate loss
            update model parameters
        for step in max_steps:
            train(SVAE)
        for step in max_steps:
            train(discriminator)
```<br><br>
To do:
 - [ ] 

In [297]:
class Solver:
    def __init__(self):
        self.epochs = 10
        self.vae_steps = 10
        self.discriminator_steps = 10
        
    def train(self):
        """ Performs model training """
    
        for epoch in range(self.epochs):
            print(f'\nEpoch: {epoch}')
            
            print('Train task learner')
            
            print('Train VAE')
            for step in range(self.vae_steps):
                print(f'VAE Step: {step}')
            
            print('Train Discriminator')
            for step in range(self.discriminator_steps):
                print(f'Discriminator Step: {step}')

In [298]:
slvr = Solver()
slvr.train()


Epoch: 0
Train task learner
Train VAE
VAE Step: 0
VAE Step: 1
VAE Step: 2
VAE Step: 3
VAE Step: 4
VAE Step: 5
VAE Step: 6
VAE Step: 7
VAE Step: 8
VAE Step: 9
Train Discriminator
Discriminator Step: 0
Discriminator Step: 1
Discriminator Step: 2
Discriminator Step: 3
Discriminator Step: 4
Discriminator Step: 5
Discriminator Step: 6
Discriminator Step: 7
Discriminator Step: 8
Discriminator Step: 9

Epoch: 1
Train task learner
Train VAE
VAE Step: 0
VAE Step: 1
VAE Step: 2
VAE Step: 3
VAE Step: 4
VAE Step: 5
VAE Step: 6
VAE Step: 7
VAE Step: 8
VAE Step: 9
Train Discriminator
Discriminator Step: 0
Discriminator Step: 1
Discriminator Step: 2
Discriminator Step: 3
Discriminator Step: 4
Discriminator Step: 5
Discriminator Step: 6
Discriminator Step: 7
Discriminator Step: 8
Discriminator Step: 9

Epoch: 2
Train task learner
Train VAE
VAE Step: 0
VAE Step: 1
VAE Step: 2
VAE Step: 3
VAE Step: 4
VAE Step: 5
VAE Step: 6
VAE Step: 7
VAE Step: 8
VAE Step: 9
Train Discriminator
Discriminator Step: 0
D

### Inference Routine
To do:
 - [ ] 


### Sampling Routine
To do:
 - [ ] 